## Sandbox for marching squares

In [221]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from marching_squares import Grid, UnitCell, Lattice
from tables import STATES

In [222]:
def center_ellipse(x,y,r,c):

    draw.ellipse([x - r, y - r, x + r, y + r],fill=c)

def center_rectangle(x,y,l,w,c):
    l = l/2
    w = w/2
    draw.rectangle([x - w, y - l, x + w, y + l],fill=c)

Marching squares

In [223]:
def get_state(a,b,c,d,threshold):
    
    # Either 0 or 1
    # < threshold = 0
    # > threshold = 1

    a = threshold + a > threshold
    b = threshold + b > threshold
    c = threshold + c > threshold
    d = threshold + d > threshold

    # Index for lookup table
    s = round(a * 8 + b * 4 + c * 2 + d * 1)

    return s

def circle(x, y, cx, cy, radius):
    # distance from center point
    d = np.sqrt((cx-x) ** 2 + (cy-y) ** 2) - radius
    return d


In [224]:
def lerp(a, b, t):
    c = a + t * (b - a)
    return c

def lerp_points(p0, p1, t):

    return [
        lerp(p0[0], p1[0], t),
        lerp(p0[1], p1[1], t)
    ]

def find_t(a, b, iso):
    t = (iso - a) / (b - a)
    # (iso_val - v0) / (v1 - v0)
    return max(min(t, 1), -1)

def find_lerp_factor(v0, v1, iso_val):
    return (iso_val - v0) / (v1 - v0)
# pub fn find_lerp_factor(v0: f64, v1: f64, iso_val: f64) -> f64 {
#     (iso_val - v0) / (v1 - v0)
# }


In [225]:
image_resolution = 1080

img = Image.new('RGB', (image_resolution, image_resolution))
draw = ImageDraw.Draw(img)

grid_divisions = 10
grid_scale = image_resolution / (grid_divisions - 1)

grid = Grid(grid_scale, grid_divisions, grid_divisions)

cx = image_resolution / 2.0
cy = image_resolution / 2.0
center = (cx, cy)
radius = image_resolution / 3
iso = 0
interpolated = False

# Drawing shapes 
center_rectangle(image_resolution / 2, image_resolution / 2, image_resolution, image_resolution, f'rgb({73},{73},{71})')
center_ellipse(image_resolution / 2, image_resolution / 2, radius, f'rgb({68},{204},{255})')


for point in grid.points:
    # distance value of current point
    dist = circle(point[0], point[1], *center, radius)

    scale = grid.scale

    x = point[0]
    y = point[1]

    ## Corner points
    p0 = (x        , y        )
    p1 = (x + scale, y        )
    p2 = (x + scale, y + scale)
    p3 = (x        , y + scale)

    ## values at (corner) points
    v0 = circle(*p0, *center, radius)
    v1 = circle(*p1, *center, radius)
    v2 = circle(*p2, *center, radius)
    v3 = circle(*p3, *center, radius)

    corner_values = [v0, v1, v2, v3]

    ## Interopolation factors
    ta = find_lerp_factor(v0, v1, iso)
    tb = find_lerp_factor(v1, v2, iso)
    tc = find_lerp_factor(v3, v2, iso) # flip due to sign change
    td = find_lerp_factor(v0, v3, iso)

    ## edge point locations (interpolated)
    a = [x + ta * scale , y              ] # 01
    b = [x + scale      , y + tb * scale ] # 12
    c = [x + tc * scale , y + scale      ] # 23
    d = [x              , y + td * scale ] # 30

    ## edge point locations (not interpolated)
    _a = [x + 0.5 * scale , y              ] # 01
    _b = [x + scale      , y + 0.5 * scale ] # 12
    _c = [x + 0.5 * scale , y + scale      ] # 23
    _d = [x              , y + 0.5 * scale ] # 30


    edge_points = [a,b,c,d]
    if interpolated == False:
        edge_points = [_a,_b,_c,_d]
    state = get_state(v0, v1, v2, v3, 0)
    edges = STATES[state]
    
    for line in edges:
        p1 = edge_points[line[0]]
        p2 = edge_points[line[1]]

        draw.line([p1[0], p1[1], p2[0], p2[1]], fill=f'rgb({255},{255},{255})',width=4)

    if dist < 0:
        center_ellipse(point[0], point[1], 2, f'rgb({53},{255},{105})')
    else:
        center_ellipse(point[0], point[1], 2, f'rgb({209},{56},{191})')

img.save('marched2d.png')
# img.show()

/var/folders/rb/9_d3yfrd1qg18m0yts9lc8gw0000gn/T/ipykernel_5065/43390966.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  return (iso_val - v0) / (v1 - v0)


In [226]:
def map(x, y):
    d = circle(x, y, *center, radius)
    return d

In [248]:
image_resolution = 1080
cx = image_resolution / 2.0
cy = image_resolution / 2.0
center = (cx, cy)
radius = image_resolution / 3
iso = 0
interpolated = True
grid_divisions = 50

Function version

In [249]:
img = Image.new('RGB', (image_resolution, image_resolution))
draw = ImageDraw.Draw(img)

grid_scale = image_resolution / (grid_divisions - 1)

grid = Grid(grid_scale, grid_divisions, grid_divisions)

graph = []

## Drawing shapes 
# Background
center_rectangle(image_resolution / 2, image_resolution / 2, image_resolution, image_resolution, f'rgb({73},{73},{71})')
## Circle (for map(p))
center_ellipse(image_resolution / 2, image_resolution / 2, radius, f'rgb({68},{204},{255})')


def march(grid, draw_points):

    for point in grid.points:
        # distance value of current point
        dist = map(*point)

        scale = grid.scale

        x = point[0]
        y = point[1]

        ## Corner points
        p0 = (x        , y        )
        p1 = (x + scale, y        )
        p2 = (x + scale, y + scale)
        p3 = (x        , y + scale)

        ## values at (corner) points
        v0 = map(*p0)
        v1 = map(*p1)
        v2 = map(*p2)
        v3 = map(*p3)

        corner_values = [v0, v1, v2, v3]

        ## Interopolation factors
        ta = find_lerp_factor(v0, v1, iso)
        tb = find_lerp_factor(v1, v2, iso)
        tc = find_lerp_factor(v3, v2, iso) # flip due to sign change
        td = find_lerp_factor(v0, v3, iso)

        ## edge point locations (interpolated)
        a = [x + ta * scale , y              ] # 01
        b = [x + scale      , y + tb * scale ] # 12
        c = [x + tc * scale , y + scale      ] # 23
        d = [x              , y + td * scale ] # 30

        ## edge point locations (not interpolated)
        _a = [x + 0.5 * scale , y               ] # 01
        _b = [x + scale       , y + 0.5 * scale ] # 12
        _c = [x + 0.5 * scale , y + scale       ] # 23
        _d = [x               , y + 0.5 * scale ] # 30


        edge_points = [a,b,c,d]
        if interpolated == False:
            edge_points = [_a,_b,_c,_d]
        state = get_state(v0, v1, v2, v3, 0)
        edges = STATES[state]
        

        for line in edges:
            p1 = edge_points[line[0]]
            p2 = edge_points[line[1]]
            graph.append((p1, p2))

        if draw_points:
            if dist < 0:
                center_ellipse(point[0], point[1], 2, f'rgb({53},{255},{105})')
            else:
                center_ellipse(point[0], point[1], 2, f'rgb({209},{56},{191})')

    return graph

def draw_edges(grid, img_path):

    edges = march(grid, True)

    for line in edges:
        p1 = line[0]
        p2 = line[1]
        draw.line([p1[0], p1[1], p2[0], p2[1]], fill=f'rgb({255},{255},{255})',width=4)
    
    img.save(img_path)

draw_edges(grid, "marched2d.png")

/var/folders/rb/9_d3yfrd1qg18m0yts9lc8gw0000gn/T/ipykernel_5065/43390966.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  return (iso_val - v0) / (v1 - v0)
